In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        
        title_value = title.text
        
        title_string = title_value.strip()
        
    except AttributeError:
        title_string = ""
        
    return title_string

In [12]:
# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()
    
    except AttributeError:
        
        try:
            price = soup.find("span", attrs={'class':'offscreen'}).string.strip()
            
        except:
            price = ""
            
    return price                

In [4]:
# Function to extract Product Rating
def get_rating(soup):
 
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
         
    except AttributeError:
         
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = "" 
 
    return rating

In [5]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
         
    except AttributeError:
        review_count = ""   
 
    return review_count

In [6]:
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()
 
    except AttributeError:
        available = "Not Available"
 
    return available 

In [18]:
if __name__ == '__main__':
    
    # Headers for permission from website
    HEADERS = ({"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.5"})
    
    URL = "https://www.amazon.ca/s?k=apple+iphone+unlocked&i=electronics&rh=n%3A3379583011%2Cp_89%3AApple%2Cp_n_feature_sixteen_browse-bin%3A24424947011&dc&qid=1678606769&rnid=24424943011&ref=sr_nr_p_ru_0&ds=v1%3ARt%2F%2FNT1r9uXLKFEpZ6wCL9GrKVHzhfuEzfDtmMTlyFg"
    
    # HTTP requests
    webpage = requests.get(URL, headers = HEADERS)
    
    # Soup object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Fetch links as List of Tag Objects
    links = soup.find_all('a', attrs={'class':'a-link-normal s-no-outline'})
    
    links_list = []
    
    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))
    
    d = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}
    
    # Loop for extracting details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers = HEADERS)
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [19]:
amazon_df

,title,price,rating,reviews,availability
0,"Apple iPhone 12 Pro Max 5G, US Version, 128GB,...",$594.99,4.3 out of 5 stars,"2,535 ratings",Not Available
1,"Apple iPhone 12, 64GB, Purple - Unlocked (Rene...",$419.99,4.3 out of 5 stars,"6,514 ratings",Only 20 left in stock - order soon.
2,"Apple iPhone 13 Pro Max, 128GB, Sierra Blue - ...",$989.89,4.4 out of 5 stars,994 ratings,Only 2 left in stock - order soon.
3,"Apple iPhone 13 Mini, 128GB, Pink - Unlocked (...",$484.00,4.5 out of 5 stars,933 ratings,Not Available
4,"Apple iPhone 13, 128GB, Pink - Unlocked (Renewed)",$620.00,4.4 out of 5 stars,"1,433 ratings",Only 2 left in stock - order soon.
5,"Apple iPhone 12, 64GB, Green - Fully Unlocked ...",$406.75,4.3 out of 5 stars,"6,515 ratings",
6,"Apple iPhone 12, 128GB, Black - Fully Unlocked...",$442.55,4.3 out of 5 stars,"6,515 ratings",Not Available
7,"Apple iPhone 12, 64GB, White - Fully Unlocked ...",$406.75,4.3 out of 5 stars,"6,514 ratings",Only 1 left in stock - order soon.
8,"Apple iPhone 13, 128GB, Midnight - Unlocked (R...",$639.99,4.4 out of 5 stars,"1,433 ratings",
9,"Apple iPhone 13 Mini, 128GB, Blue - Unlocked (...",$522.92,4.5 out of 5 stars,933 ratings,Only 1 left in stock - order soon.
